In [1]:
!pip install simpletransformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 315.5/315.5 kB 5.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 10.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 5.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.7/101.7 kB 11.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 19.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.1/8.1 MB 45.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 30.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 266.1/266.1 kB 33.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 18.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 25.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 19.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB

In [2]:
import json
with open(r'train.json','r') as read_file:
  train=json.load(read_file)

In [3]:
import json
with open(r'test.json','r') as read_file:
  test=json.load(read_file)

In [4]:
import json
with open(r'predictions.json','r') as read_file:
  predictions=json.load(read_file)

In [5]:
import logging

from simpletransformers.question_answering import QuestionAnsweringModel, QuestionAnsweringArgs

In [6]:
## Showing the list of model names and model types
model_type='bert'
model_name='bert-base-cased'

if model_type=='bert':
  model_name='bert-base-cased'

elif model_type=='roberta':
  model_name='roberta-base'

elif model_type=='distilbert':
  model_name='distilbert-base-cased'

elif model_type=='distilroberta':
  model_type='roberta'
  model_name='distilroberta-base'

elif model_type=='electra-base':
  model_type='electra'
  model_name='google/electra-base-discriminator'

elif model_type=='electra-small':
  model_type='electra'
  model_name='google/electra-small-discriminator'



In [7]:
## Model configuration: setting up parameters
model_args = QuestionAnsweringArgs()
model_args.train_batch_size = 16
model_args.evaluate_during_training = True
model_args.n_best_size=3
model_args.num_train_epochs=5


In [8]:

### Advanced Methodology
train_args = {
    "reprocess_input_data": True,
    "overwrite_output_dir": True,
    "use_cached_eval_features": True,
    "output_dir": f"outputs/{model_type}",
    "best_model_dir": f"outputs/{model_type}/best_model",
    "evaluate_during_training": True,
    "max_seq_length": 128,
    "num_train_epochs": 100,
    "evaluate_during_training_steps": 100,
    # "wandb_project": "Question Answer Application",
    # "wandb_kwargs": {"name": model_name},
    "save_model_every_epoch": False,
    "save_eval_checkpoints": False,
    "n_best_size":3,
    # "use_early_stopping": True,
    # "early_stopping_metric": "mcc",
    # "n_gpu": 2,
    # "manual_seed": 4,
    # "use_multiprocessing": False,
    "train_batch_size": 128,
    "eval_batch_size": 64,
    # "config": {
    #     "output_hidden_states": True
    # }
}

In [9]:
## Initializing the model

model = QuestionAnsweringModel(
    model_type,model_name, args=train_args
)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/436M [00:00<?, ?B/s]

Some weights of BertForQuestionAnswering were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/436k [00:00<?, ?B/s]

In [10]:
# train
## for every context extract all answers, append it to context, look to update the 'answer_start' parameter

# contexts=[]
# answers=[]

for item in train:
  ctx=item['context']
  for ques_ans in item['qas']:
    for ans_list in ques_ans['answers']:
      ans_list['answer_start']=len(ctx)-1
      ctx=ctx+ans_list['text']

  item['context']=ctx

# train






In [11]:
for item in test:
  ctx=item['context']
  for ques_ans in item['qas']:
    for ans_list in ques_ans['answers']:
      ans_list['answer_start']=len(ctx)-1
      ctx=ctx+ans_list['text']

  item['context']=ctx



In [12]:
for item in predictions:
  ctx=item['context']
  for ques_ans in item['qas']:
    for ans_list in ques_ans['answers']:
      ans_list['answer_start']=len(ctx)-1
      ctx=ctx+ans_list['text']

  item['context']=ctx

In [13]:
model.train_model(train, eval_data=test)


add example index and unique id: 100%|██████████| 54/54 [00:00<00:00, 313267.52it/s]


Epoch:   0%|          | 0/100 [00:00<?, ?it/s]

Running Epoch 1 of 100:   0%|          | 0/1 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/torch/optim/lr_scheduler.py:143: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  warnings.warn("Detected call of `lr_scheduler.step()` before `optimizer.step()`. "

convert squad examples to features: 100%|██████████| 19/19 [00:00<00:00, 195.24it/s]

add example index and unique id: 100%|██████████| 19/19 [00:00<00:00, 165679.37it/s]


Running Evaluation:   0%|          | 0/1 [00:00<?, ?it/s]

...

Running Evaluation:   0%|          | 0/1 [00:00<?, ?it/s]

Running Epoch 100 of 100:   0%|          | 0/1 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/1 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/1 [00:00<?, ?it/s]

(100,
 {'global_step': [1,
   2,
   3,
   4,
   5,
   6,
   7,
   8,
   9,
   10,
   11,
   12,
   13,
   14,
   15,
   16,
   17,
   18,
   19,
   20,
   21,
   22,
   23,
   24,
   25,
   26,
   27,
   28,
   29,
   30,
   31,
   32,
   33,
   34,
   35,
   36,
   37,
   38,
   39,
   40,
   41,
   42,
   43,
   44,
   45,
   46,
   47,
   48,
   49,
   50,
   51,
   52,
   53,
   54,
   55,
   56,
   57,
   58,
   59,
   60,
   61,
   62,
   63,
   64,
   65,
   66,
   67,
   68,
   69,
   70,
   71,
   72,
   73,
   74,
   75,
   76,
   77,
   78,
   79,
   80,
   81,
   82,
   83,
   84,
   85,
   86,
   87,
   88,
   89,
   90,
   91,
   92,
   93,
   94,
   95,
   96,
   97,
   98,
   99,
   100,
   100],
  'correct': [0,
   0,
   0,
   0,
   8,
   11,
   8,
   8,
   15,
   14,
   9,
   7,
   10,
   19,
   19,
   19,
   19,
   19,
   19,
   19,
   19,
   19,
   19,
   19,
   19,
   19,
   19,
   19,
   19,
   19,
   19,
   19,
   19,
   19,
   19,
   19,
   19,
   19,
   19,
   

In [14]:
result,texts=model.eval_model(test)

Running Evaluation:   0%|          | 0/1 [00:00<?, ?it/s]

In [15]:
to_predict_1=[
   {'context': 'The batsman is facing difficulty against the off-spin deliveries turning into them.The batsman should watch the ball closely, play with soft hands, and look to use their feet to disrupt the length and adjust to the turn.',
  'qas': [
      {'id': '00021',
           'question': 'How should the batsman tackle off spin?'}
  ]
   }
]


to_predict_2=[
   {'context': 'Batsman has been edging the ball quite often. It shows eagerness to get off the mark. It is advisable leave a lot of balls outside off stump, atleast at the beginning. The ball will stop swinging and it becomes easier to bat. ',
  'qas': [
      {'id': '00022',
           'question': 'How should  batsman overcome this tendency to edge the ball?'}
  ]
   }
]


to_predict_3=[
   {'context': 'Australia are currently 190/1 in 12 overs and this is mind boggling in a t20 game. At the same time this is when Australia does have a chance of losing momentum. So India should look to approach the game by slowing down the game by introducing a spinner and look to avoid conceding boundaries. A collapse could be round the corner and this has happened to many teams before',
  'qas': [
      {'id': '00023',
           'question': 'How should India approach the game now?'}
  ]
   }
]


to_predict_4=[
   {'context': 'The wicket is flat and is offering absolutely no help for the bowlers which makes it easy for south african batsmen. South Africa have an amazing record on flat wickets and absolutely steam roll teams. However Australia is the one team which has been able to have an edge over south africa. Why, because Australia is tactically a step ahead of south africa as they plan their strategies better.',
  'qas': [
      {'id': '00024',
           'question': 'Why australia is successful over south africa?'}
  ]
   }
]


to_predict_5=[
   {'context': 'Ravichandran Ashwin is not used to being on the receiving end, (his record speaks off this fact). But fortune favours the brave.Greate players take the attack to the opposition by looking to attack them and not allow them to settle. The key to dominating ashwin is to try and attack him and disrupt his trajectory.',
  'qas': [
      {'id': '00025',
           'question': 'How should a batsman counter Ashwin?'}
  ]
   }
]



# to_predict_6=[
#    {'context': 'Ravichandran Ashwin is not used to being on the receiving end, (his record speaks off this fact). But fortune favours the brave.Greate players take the attack to the opposition by looking to attack them and not allow them to settle. The key to dominating ashwin is to try and attack him and disrupt his trajectory.',
#   'qas': [
#       {'id': '00025',
#            'question': 'How to dominate ashwin?'}
#   ]
#    }
# ]




In [16]:
ans1,prob1=model.predict(to_predict_1)
ans2,prob2=model.predict(to_predict_2)
ans3,prob3=model.predict(to_predict_3)
ans4,prob4=model.predict(to_predict_4)
ans5,prob5=model.predict(to_predict_5)
# ans6,prob6=model.predict(to_predict_6)

add example index and unique id: 100%|██████████| 1/1 [00:00<00:00, 12595.51it/s]


Running Prediction:   0%|          | 0/1 [00:00<?, ?it/s]

add example index and unique id: 100%|██████████| 1/1 [00:00<00:00, 13530.01it/s]


Running Prediction:   0%|          | 0/1 [00:00<?, ?it/s]

add example index and unique id: 100%|██████████| 1/1 [00:00<00:00, 12300.01it/s]


Running Prediction:   0%|          | 0/1 [00:00<?, ?it/s]

add example index and unique id: 100%|██████████| 1/1 [00:00<00:00, 11915.64it/s]


Running Prediction:   0%|          | 0/1 [00:00<?, ?it/s]

add example index and unique id: 100%|██████████| 1/1 [00:00<00:00, 8719.97it/s]


Running Prediction:   0%|          | 0/1 [00:00<?, ?it/s]

In [17]:
print(ans1)
print(ans2)
print(ans3)
print(ans4)
print(ans5)
#print(ans6)



[{'id': '00021', 'answer': ['The batsman should watch the ball closely, play with soft hands, and look to use their feet to disrupt the length and adjust to the turn.', '.', 'The']}]
[{'id': '00022', 'answer': ['', '.']}]
[{'id': '00023', 'answer': ['So India should look to approach the game by slowing down the game by introducing a spinner and look to avoid conceding boundaries.', '.', '. So India should look to approach the game by slowing down the game by introducing a spinner and look to avoid conceding boundaries.']}]
[{'id': '00024', 'answer': ['.', '. However Australia is the one team which has been able to have an edge over south africa. Why, because Australia is tactically a step ahead of south africa as they plan their strategies better.', '. Why, because Australia is tactically a step ahead of south africa as they plan their strategies better.']}]
[{'id': '00025', 'answer': ['. But fortune favours the brave.Greate players take the attack to the opposition by looking to attac

In [18]:
!pip install flask-ngrok

In [19]:
!pip install pyngrok
from pyngrok import ngrok

ngrok.set_auth_token("2eGg2U9gPwzN7W2CdLhg6skZF5w_2E1GnbgQ8mJ4nWZRC7fyQ")

In [ ]:

from flask import Flask,request,jsonify
from flask_ngrok import run_with_ngrok

# from pyngrok import ngrok
app = Flask(__name__)
run_with_ngrok(app)
@app.route('/', methods=['POST','GET'])
def answer_question():
    if request.method=='POST':
      data = request.get_json()
      # print(data)
      # print(data.keys())
      print(data['context'])
      print(data['question'])
      # context = data['context']
      # question = data['question']
      q=[{
          'context':data['context'],
           'qas': [
      {'id': '00025',
           'question': data['question']}
  ]
      }]
      answer,prob = model.predict(q)
      print(answer)
      anslist=answer[0]['answer']
      ans=''
      for item in anslist:
        if item!='':
          ans=ans+item
      print(ans)
      return jsonify({'answer': ans})
    if request.method=='GET':
      return 'hi'

if __name__ == '__main__':
    ngrok_tunnel = ngrok.connect(5000)
    print('Public URL:', ngrok_tunnel.public_url)
    run_with_ngrok(app)
    app.run()



Public URL: https://d135-34-125-240-100.ngrok-free.app
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit


 * Running on http://d135-34-125-240-100.ngrok-free.app
 * Traffic stats available on http://127.0.0.1:4040
 * Running on http://d135-34-125-240-100.ngrok-free.app
 * Traffic stats available on http://127.0.0.1:4040
The batsman is facing difficulty against the off-spin deliveries turning into them.The batsman should watch the ball closely, play with soft hands, and look to use their feet to disrupt the length and adjust to the turn.
How should the batsman tackle off spin?


add example index and unique id: 100%|██████████| 1/1 [00:00<00:00, 13066.37it/s]


Running Prediction:   0%|          | 0/1 [00:00<?, ?it/s]

...

[{'id': '00025', 'answer': ['The batsman should watch the ball closely, play with soft hands, and look to use their feet to disrupt the length and adjust to the turn.', '.', 'The']}]
The batsman should watch the ball closely, play with soft hands, and look to use their feet to disrupt the length and adjust to the turn..The
In the powerplay teams should maximise run scoring.
What should the opposition team do now?


add example index and unique id: 100%|██████████| 1/1 [00:00<00:00, 11125.47it/s]


Running Prediction:   0%|          | 0/1 [00:00<?, ?it/s]

INFO:werkzeug:127.0.0.1 - - [04/Apr/2024 06:53:29] "POST / HTTP/1.1" 200 -


[{'id': '00025', 'answer': ['.']}]
.
